In [11]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [0]:
import cv2
import os
import numpy as np 
from skimage import io
import matplotlib.pyplot as plt
%matplotlib inline
import os
import time
import sys

In [0]:
initialize = True
net = None
files_dir = f'./drive/My Drive/yolo'
classes = None
COLORS = np.random.uniform(0, 255, size=(80, 3))

In [0]:
#to get the classes
def get_class_labels():

    class_file_name = 'classes.txt'
    class_file = os.path.join(files_dir , class_file_name) 
    f = open(class_file, 'r')
    classes = [line.strip() for line in f.readlines()]

    return classes

In [0]:
def get_output_layers(net):
    
    layer_names = net.getLayerNames()
    
    output_layers = [layer_names[i[0] - 1] for i in net.getUnconnectedOutLayers()]

    return output_layers

In [0]:
# to draw the outboxes, imp vehicles = {car , bus , truck , bike }
def draw_out_boxes(img, bbox, labels, confidence, colors=None, write_conf=False):

    global COLORS
    global classes

    if classes is None:
        classes = populate_class_labels()
    
    for i, label in enumerate(labels):

        if colors is None:
            color = COLORS[classes.index(label)]            
        else:
            color = colors[classes.index(label)]

        if write_conf:
            label += ' ' + str(format(confidence[i] * 100, '.2f')) + '%'

        cv2.rectangle(img, (bbox[i][0],bbox[i][1]), (bbox[i][2],bbox[i][3]), color, 2)

        cv2.putText(img, label, (bbox[i][0],bbox[i][1]-10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

    return img

In [0]:
def detect_object(image, confidence=0.5, nms_thresh=0.3, model='yolov3', enable_gpu=False):

    Height, Width = image.shape[:2]
    scale = 0.00392

    global classes
    global dest_dir
    
    config_file_name = 'yolov3.cfg'
    weights_file_name = 'yolov3.weights'
    config_file = os.path.join( files_dir , config_file_name)
    weights_file = os.path.join( files_dir , weights_file_name)
    
    blob = cv2.dnn.blobFromImage(image, scale, (416,416), (0,0,0), True, crop=False) 
    
    global initialize
    global net
    
    if initialize:
        classes = get_class_labels()
        net = cv2.dnn.readNet(weights_file, config_file)
        initialize = False

    if enable_gpu:
        net.setPreferableBackend(cv2.dnn.DNN_BACKEND_CUDA)
        net.setPreferableTarget(cv2.dnn.DNN_TARGET_CUDA)

    net.setInput(blob)
    
    outs = net.forward(get_output_layers(net))

    class_numbers= []
    confidences = []
    boxes = []

    for out in outs:
        for detection in out:
            scores = detection[5:]
            class_id = np.argmax(scores)
            max_conf = scores[class_id]
            if max_conf > confidence:
                center_x = int(detection[0] * Width)
                center_y = int(detection[1] * Height)
                w = int(detection[2] * Width)
                h = int(detection[3] * Height)
                x = center_x - (w / 2)
                y = center_y - (h / 2)
                class_numbers.append(class_id)
                confidences.append(float(max_conf))
                boxes.append([x, y, w, h])
                
    indices = cv2.dnn.NMSBoxes(boxes, confidences, confidence, nms_thresh)

    bbox = []
    label = []
    conf = []

    for i in indices:
        i = i[0]
        box = boxes[i]
        x = box[0]
        y = box[1]
        w = box[2]
        h = box[3]
        bbox.append([int(x), int(y), int(x+w), int(y+h)])
        label.append(str(classes[class_numbers[i]]))
        conf.append(confidences[i])
        
    return bbox, label, conf



In [0]:
input_path =f'./drive/My Drive/Laal batti'
output_path  = f'./drive/My Drive/output_images1'
img_cnt = 0

In [0]:
# imp variables
cnt = 1
cnt1 = 0
m = int(70)
# minimum limit of maximum time
T = 70
l = 0
p = 0

def profit_time(T , len_green_time):
    p = T - len_green_time
    return p

def max_time(T , p):
    l = T  + p
    return l

def calculate_timings(vehicle_count , Maximum_time):
    # time of green signal = t
    y = 3 * vehicle_count
    if vehicle_count < 5:
        t = 10
    else:
        if y > Maximum_time:
            t= Maximum_time
        else:
            t = y
    return t    

In [0]:
for filename in os.listdir(input_path):
    if filename.endswith(".jpg") or filename.endswith(".png"):
        N = 0
        car = 0
        bus = 0
        truck = 0
        bike = 0
        img_file=os.path.join(input_path, filename) 
        original_img = io.imread(img_file)
        original_img = cv2.cvtColor(original_img, cv2.COLOR_BGR2RGB)
        img=cv2.resize(original_img,(416,416))
        
        box, label, conf = detect_object(img)
        output_image = draw_out_boxes(img, box, label, conf)
        cv2.imwrite(os.path.join(output_path , 'out'+ str(img_cnt) + '.jpg'), output_image)
        img_cnt+=1
        for element in label:
            if element == "car":
                car = car+1
            elif element == "bus":
                bus = bus +1
            elif element == "truck":
                truck = truck + 1
            elif element == "motorcycle":
                bike = bike+1
        N = car + bus + truck
        
        x = cnt - cnt1
        if x ==1:
            l = max_time(T,p)
            n = calculate_timings(N ,l)
            t1 = n
            t2 = n
            t3 = 2*T
            t4 = 3*T
            for i in range(1):
                print("len2 - red - " , t2)
                print("len3 - red - " , t3)
                print("len4 - red - " , t4)
            for remaining in range(t1, 0, -1):
                sys.stdout.write("\r")
                sys.stdout.write(" len1 green{:2d} seconds remaining.".format(remaining))
                sys.stdout.flush()
                time.sleep(1)

            m = n
            p = profit_time(l, m)
            if p>T:
                p = T
            else:
                p = p
            cnt += 1
            print("\n")
    
        if x == 2:
            l = max_time(T , p)
            n = calculate_timings(N , T+p)
            t1 = 3*T
            t2 = n
            t3 = n
            t4 = (3*T - m)
            for i in range(1):
                print("len1 - red - " , t1)
                print("len3 - red - " , t3)
                print("len4 - red - " , t4)
            for remaining in range(t2, 0, -1):
                sys.stdout.write("\r")
                sys.stdout.write(" len2 green {:2d} seconds remaining.".format(remaining))
                sys.stdout.flush()
                time.sleep(1)
            m = n
            p = profit_time(l, m)
            if p>T:
                p = T
            else:
                p = p
            cnt += 1
            print("\n")
    
        if x == 3:
            l = max_time(T , p)
            n = calculate_timings(N , l)
            t1 = (3*T - m)
            t2 = 3*T
            t3 = n
            t4 = n
            for i in range(1):
                print("len1 - red - " , t1)
                print("len2 - red - " , t2)
                print("len4 - red - " , t4)

            for remaining in range(t3, 0, -1):
                sys.stdout.write("\r")
                sys.stdout.write(" len3 green{:2d} seconds remaining.".format(remaining))
                sys.stdout.flush()
                time.sleep(1)

        
            m = n
            p = profit_time(l , m)
            if p>T:
                p = T
            else:
                p = p
            cnt += 1
            print("\n")
    
        if x == 4:
            l = max_time(T , p)
            n = calculate_timings(N , l)
            t1 = n
            t2 = (3*T - m)
            t3 = 3*T
            t4 = n
            for i in range(1):
                print("len1 - red - " , t1)
                print("len2 - red - " , t2)
                print("len3 - red - " , t3)
            for remaining in range(t4, 0, -1):
                sys.stdout.write("\r")
                sys.stdout.write(" len4 green{:2d} seconds remaining.".format(remaining))
                sys.stdout.flush()
                time.sleep(1)

            m = n
            p = profit_time(l , m)
            if p>T:
                p = T
            else:
                p = p
            cnt= 1
            print("\n")
            
    else:
        continue

len2 - red -  70
len3 - red -  140
len4 - red -  210
 len1 green 1 seconds remaining.

len1 - red -  210
len3 - red -  70
len4 - red -  140
 len2 green  1 seconds remaining.

len1 - red -  140
len2 - red -  210
len4 - red -  63
 len3 green 1 seconds remaining.

len1 - red -  77
len2 - red -  147
len3 - red -  210
 len4 green 1 seconds remaining.

len2 - red -  70
len3 - red -  140
len4 - red -  210
 len1 green66 seconds remaining.